# For colab environment

In [1]:
# !pip install nltk transformers==4.35.0 torch==2.6.0 torchvision==0.21.0 datasets accelerate==0.24.0 huggingface==0.0.1 datasets==2.14.7

In [2]:
import torch 
print(torch.cuda.is_available())
print(torch.__version__)

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/vol/bitbucket/bm1325/dl_cw_1/dlvenv/lib/python3.12/site-packages/traitlets/traitlets.py", line 632, in get
    value = obj._trait_values[self.name]
            ~~~~~~~~~~~~~~~~~^^^^^^^^^^^
KeyError: '_control_lock'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/vol/bitbucket/bm1325/dl_cw_1/dlvenv/lib/python3.12/site-packages/zmq/eventloop/zmqstream.py", line 565, in _log_error
    f.result()
  File "/vol/bitbucket/bm1325/dl_cw_1/dlvenv/lib/python3.12/site-packages/ipykernel/kernelbase.py", line 340, in dispatch_control
    async with self._control_lock:
               ^^^^^^^^^^^^^^^^^^
  File "/vol/bitbucket/bm1325/dl_cw_1/dlvenv/lib/python3.12/site-packages/traitlets/traitlets.py", line 687, in __get__
    return t.cast(G, self.get(obj, cls))  # the G should encode the Optional
                     ^^^^^^^^^^^^^^

True
2.6.0+cu124


In [3]:
# !git clone https://github.com/BernardMoy/NLP-PCL-Classification.git

In [4]:
# %cd NLP-PCL-Classification/

In [5]:
!nvidia-smi

Sun Mar  1 12:04:38 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.163.01             Driver Version: 550.163.01     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA TITAN Xp                Off |   00000000:02:00.0  On |                  N/A |
| 25%   41C    P8             19W /  250W |     362MiB /  12288MiB |     25%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Load train and validation data set

In [6]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt
from collections import Counter

df = pd.read_csv('data/dontpatronizeme_pcl.tsv', sep='\t')

# Remove rows with NA labels 
df = df.dropna() 

# Add a bool_labels column for binary classification
df["bool_labels"] = df["label"] > 1   # is PCL if >1

# train val split 
train_labels = pd.read_csv('data/train_semeval_parids-labels.csv')["par_id"]
val_labels = pd.read_csv('data/dev_semeval_parids-labels.csv')["par_id"]
df_train = df[df["par_id"].isin(train_labels)]
df_val = df[df["par_id"].isin(val_labels)]


# Tokenization

In [7]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AutoConfig, Trainer, TrainingArguments

tokenizer = RobertaTokenizer.from_pretrained("roberta-base") 

# Create text with contextual information 
def tokenize(df): 
    text_with_context = df["text"] # df["keyword"] + "</s><s>" + df["country_code"] + "</s><s>" + df["text"]

    encoding = tokenizer(
        text_with_context.tolist(), 
        padding="max_length",   # Add padding to shorter sentences 
        max_length=256,
        truncation = True, 
        return_attention_mask = True 
    )

    return encoding

/vol/bitbucket/bm1325/dl_cw_1/dlvenv/lib/python3.12/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/vol/bitbucket/bm1325/dl_cw_1/dlvenv/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/vol/bitbucket/bm1325/dl_cw_1/dlvenv/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


# Convert to pyTorch dataset

In [8]:
import torch 
from torch.utils.data import DataLoader, TensorDataset
from datasets import Dataset

def to_dataset(df): 
    # Obtain tokens (input_ids, attention_mask) from the dataset 
    encoding = tokenize(df) 

    # Return huggingface dataset 
    return Dataset.from_dict({
        "input_ids": encoding["input_ids"], 
        "attention_mask": encoding["attention_mask"], 
        "label": df["bool_labels"].values 
    })

In [9]:
train_dataset = to_dataset(df_train)
val_dataset = to_dataset(df_val) 

# set to torch format 
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# Training 

In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=-1)

    # Calculate metrics 
    accuracy = accuracy_score(labels, predictions) 
    precision = precision_score(labels, predictions) 
    recall = recall_score(labels, predictions) 
    f1 = f1_score(labels, predictions) 

    return {
        "accuracy": accuracy, 
        "precision": precision, 
        "recall": recall, 
        "f1": f1 
    }


In [ ]:
# Load roberta sequence classification model 
config = AutoConfig.from_pretrained("roberta-base", num_labels=2)  # Binary classification
model = RobertaForSequenceClassification.from_pretrained("roberta-base", config = config)

BATCH_SIZE = 32
N_EPOCHS = 5 

# Set up training arguments 
training_args = TrainingArguments(
    fp16=True, 
    num_train_epochs=N_EPOCHS, 
    learning_rate=2e-5, 
    weight_decay=0.01,
    warmup_steps=500, 
    save_strategy="epoch",  # low disk space 
    load_best_model_at_end=True, 
    metric_for_best_model='f1',
    logging_steps=50,
    output_dir="./checkpoints/baseline", 
    evaluation_strategy="epoch", 
    per_device_eval_batch_size=BATCH_SIZE, 
    per_device_train_batch_size=BATCH_SIZE, 
)

# Set up trainer 
trainer = Trainer(
    model = model, 
    args = training_args, 
    train_dataset=train_dataset, 
    eval_dataset=val_dataset, 
    compute_metrics=compute_metrics
)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/vol/bitbucket/bm1325/dl_cw_1/dlvenv/lib/python3.12/site-packages/accelerate/accelerator.py:439: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [12]:
trainer.train() 

  0%|          | 0/1310 [00:00<?, ?it/s]

{'loss': 0.6206, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.19}
{'loss': 0.3812, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.38}
{'loss': 0.3112, 'learning_rate': 6e-06, 'epoch': 0.57}
{'loss': 0.2478, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.76}
{'loss': 0.2281, 'learning_rate': 9.960000000000001e-06, 'epoch': 0.95}


  0%|          | 0/66 [00:00<?, ?it/s]

{'eval_loss': 0.21420791745185852, 'eval_accuracy': 0.915910176779742, 'eval_precision': 0.6138613861386139, 'eval_recall': 0.31155778894472363, 'eval_f1': 0.41333333333333333, 'eval_runtime': 17.5353, 'eval_samples_per_second': 119.359, 'eval_steps_per_second': 3.764, 'epoch': 1.0}
{'loss': 0.1938, 'learning_rate': 1.196e-05, 'epoch': 1.15}
{'loss': 0.2405, 'learning_rate': 1.396e-05, 'epoch': 1.34}
{'loss': 0.2234, 'learning_rate': 1.5960000000000003e-05, 'epoch': 1.53}
{'loss': 0.1918, 'learning_rate': 1.796e-05, 'epoch': 1.72}
{'loss': 0.1925, 'learning_rate': 1.9920000000000002e-05, 'epoch': 1.91}


  0%|          | 0/66 [00:00<?, ?it/s]

...

  0%|          | 0/66 [00:00<?, ?it/s]

{'eval_loss': 0.32617151737213135, 'eval_accuracy': 0.9096989966555183, 'eval_precision': 0.5204918032786885, 'eval_recall': 0.6381909547738693, 'eval_f1': 0.5733634311512416, 'eval_runtime': 17.4996, 'eval_samples_per_second': 119.603, 'eval_steps_per_second': 3.772, 'epoch': 4.0}
{'loss': 0.0807, 'learning_rate': 6.4691358024691365e-06, 'epoch': 4.01}
{'loss': 0.0422, 'learning_rate': 5.234567901234568e-06, 'epoch': 4.2}
{'loss': 0.0466, 'learning_rate': 4.000000000000001e-06, 'epoch': 4.39}
{'loss': 0.0219, 'learning_rate': 2.765432098765432e-06, 'epoch': 4.58}
{'loss': 0.0572, 'learning_rate': 1.5555555555555558e-06, 'epoch': 4.77}
{'loss': 0.0326, 'learning_rate': 3.209876543209877e-07, 'epoch': 4.96}


  0%|          | 0/66 [00:00<?, ?it/s]

{'eval_loss': 0.3859855830669403, 'eval_accuracy': 0.9202102245580507, 'eval_precision': 0.5816326530612245, 'eval_recall': 0.5728643216080402, 'eval_f1': 0.5772151898734177, 'eval_runtime': 17.3333, 'eval_samples_per_second': 120.75, 'eval_steps_per_second': 3.808, 'epoch': 5.0}
{'train_runtime': 1140.0652, 'train_samples_per_second': 36.73, 'train_steps_per_second': 1.149, 'train_loss': 0.16433913120786653, 'epoch': 5.0}


TrainOutput(global_step=1310, training_loss=0.16433913120786653, metrics={'train_runtime': 1140.0652, 'train_samples_per_second': 36.73, 'train_steps_per_second': 1.149, 'train_loss': 0.16433913120786653, 'epoch': 5.0})

In [13]:
trainer.evaluate()

  0%|          | 0/66 [00:00<?, ?it/s]

{'eval_loss': 0.3859855830669403,
 'eval_accuracy': 0.9202102245580507,
 'eval_precision': 0.5816326530612245,
 'eval_recall': 0.5728643216080402,
 'eval_f1': 0.5772151898734177,
 'eval_runtime': 16.7706,
 'eval_samples_per_second': 124.802,
 'eval_steps_per_second': 3.935,
 'epoch': 5.0}

# Save model

In [ ]:
trainer.save_model('models/model_baseline')

# Load model

In [15]:
# import os 
# # must pass abs path here 
# model = RobertaForSequenceClassification.from_pretrained(os.path.abspath("/vol/bitbucket/bm1325/NLP-PCL-Classification/baseline_model"), local_files_only = True)

# model